In [8]:
import os
from pathlib import Path
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import DirectoryLoader, TextLoader

In [9]:
CONFIG = {
    "data_path": "dulieu_daotao",
    "vectorstore_path": "faiss_index",
    "embedding_model": "sentence-transformers/all-MiniLM-L6-v2",
    "retriever_k": 3
}

In [10]:
# Giả sử code tạm phần data của a Quân

def prepare_vector_database(config):
    vectorstore_path = Path(config["vectorstore_path"])
    data_path = Path(config["data_path"])

    if (vectorstore_path.exists()):
        print("csdl vector đã tồn tại. Bỏ qua tạo mới")
        return
    
    if not data_path.exists():
        data_path.mkdir(parents=True, exist_ok=True)
        (data_path / "hoc_phi.txt").write_text("Quy định về học phí năm học 2024-2025 cho sinh viên ngành Công nghệ thông tin là 30 triệu đồng/năm. Sinh viên có thể đóng theo hai kỳ. Ngành Quản trị kinh doanh có mức học phí là 28 triệu đồng/năm.", encoding='utf-8')
        (data_path / "lich_nghi.txt").write_text("Trường thông báo lịch nghỉ lễ Quốc Khánh. Toàn thể sinh viên được nghỉ vào ngày 2 tháng 9. Lịch thi cuối kỳ sẽ được thông báo sau ngày nghỉ lễ.", encoding='utf-8')
        (data_path / "thu_tuc.txt").write_text("Thủ tục nhập học cho tân sinh viên yêu cầu nộp bản sao công chứng học bạ THPT và giấy báo trúng tuyển trước ngày 15 tháng 8. Thủ tục đăng ký thi lại cần hoàn thành đơn trực tuyến trên cổng thông tin.", encoding='utf-8')

    # Tải dữ liệu
    loader = DirectoryLoader(
        str(data_path),
        glob="**/*.txt",
        loader_cls=TextLoader,
        loader_kwargs={'encoding': 'utf-8'}
    )
    documents = loader.load()

    #Chia chunk
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = 500,
        chunk_overlap = 50
    )
    docs = text_splitter.split_documents(documents)
    print("Đã chia {len(documents)} tài liệu thành {len(docs)} chunks")

    embeddings_model = HuggingFaceEmbeddings(
        model_name = config["embedding_model"],
        model_kwargs = {'device' : 'cpu'}
    )

    vectorstore = FAISS.from_documents(docs, embeddings_model)
    vectorstore.save_local(config["vectorstore_path"])


In [11]:
class QuerySystem:
    def __init__(self, config):
        self.config = config

        vectorstore_path = self.config['vectorstore_path']
        if not Path(vectorstore_path).exists():
            raise FileNotFoundError(
                f'không tìm thấy thư mực CSDL vector'
            )
        
        self.embeddings_model = HuggingFaceEmbeddings(
            model_name=self.config["embedding_model"],
            model_kwargs={'device': 'cpu'}
        )

        self.vectorstore = FAISS.load_local(
            vectorstore_path,
            embeddings=self.embeddings_model,
            allow_dangerous_deserialization=True
        )

        # Tạo retriever cơ bản
        self.retriever = self.vectorstore.as_retriever(
            search_lwargs={'k': self.config["retriever_k"]}
        )

    def query(self, text: str) -> list[Document]:
        print(f"Đang thực hiện truy vấn câu hỏi: {text}")
        results = self.retriever.invoke(text)
        return results

In [12]:
# Tích hợp reranking 
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

def create_reranking_retriever(query_system: QuerySystem):
    try:
        cohere_api_key = os.environ.get("COHERE_API_KEY")
        if not cohere_api_key:
            raise ValueError("Không thấy cohere api")
        
        compressor = CohereRerank(model="rerank-multilingual-v3.0", top_n=2)

        compression_retriever = ContextualCompressionRetriever(
            base_compressor=compressor,
            base_retriever=query_system.retriever
        )
        return compression_retriever
    
    except (ImportError, ValueError) as e:
        print(f"Lỗi khi cấu hình Cohere Rerank: {e}")
        print(" => Bỏ qua.Sử dụng retriever cơ bản.")
        return None

In [13]:

prepare_vector_database(CONFIG)
Query_System = QuerySystem(CONFIG)

#Đặt câu hỏi
sample_queries = [
    "học phí ngành công nghệ thông tin là bao nhiêu?",
    "khi nào trường nghỉ lễ?",
    "thủ tục thi lại gồm những gì?"
]

for q in sample_queries:
    results = Query_System.query(q)
    print("3 vị trí sau khi truy vấn")
    for doc in results:
        print("- " + doc.page_content[:100])

csdl vector đã tồn tại. Bỏ qua tạo mới
Đang thực hiện truy vấn câu hỏi: học phí ngành công nghệ thông tin là bao nhiêu?
3 vị trí sau khi truy vấn
- Quy định về học phí năm học 2024-2025 cho sinh viên ngành Công nghệ thông tin là 30 triệu đồng/năm. 
- Trường thông báo lịch nghỉ lễ Quốc Khánh. Toàn thể sinh viên được nghỉ vào ngày 2 tháng 9. Lịch thi 
- Thủ tục nhập học cho tân sinh viên yêu cầu nộp bản sao công chứng học bạ THPT và giấy báo trúng tuyể
Đang thực hiện truy vấn câu hỏi: khi nào trường nghỉ lễ?
3 vị trí sau khi truy vấn
- Trường thông báo lịch nghỉ lễ Quốc Khánh. Toàn thể sinh viên được nghỉ vào ngày 2 tháng 9. Lịch thi 
- Thủ tục nhập học cho tân sinh viên yêu cầu nộp bản sao công chứng học bạ THPT và giấy báo trúng tuyể
- Quy định về học phí năm học 2024-2025 cho sinh viên ngành Công nghệ thông tin là 30 triệu đồng/năm. 
Đang thực hiện truy vấn câu hỏi: thủ tục thi lại gồm những gì?
3 vị trí sau khi truy vấn
- Thủ tục nhập học cho tân sinh viên yêu cầu nộp bản sao công ch

In [15]:
# Dùng reranking        
advanced_retriever = create_reranking_retriever(Query_System)
if advanced_retriever:
    for q in sample_queries:
        rr_results = advanced_retriever.invoke(q)
        print("3 vị trí sau khi truy vấn sau khi reranking")
        for doc in rr_results:
            print("- " + doc.page_content[:100])

3 vị trí sau khi truy vấn sau khi reranking
- Quy định về học phí năm học 2024-2025 cho sinh viên ngành Công nghệ thông tin là 30 triệu đồng/năm. 
- Trường thông báo lịch nghỉ lễ Quốc Khánh. Toàn thể sinh viên được nghỉ vào ngày 2 tháng 9. Lịch thi 
3 vị trí sau khi truy vấn sau khi reranking
- Trường thông báo lịch nghỉ lễ Quốc Khánh. Toàn thể sinh viên được nghỉ vào ngày 2 tháng 9. Lịch thi 
- Thủ tục nhập học cho tân sinh viên yêu cầu nộp bản sao công chứng học bạ THPT và giấy báo trúng tuyể
3 vị trí sau khi truy vấn sau khi reranking
- Thủ tục nhập học cho tân sinh viên yêu cầu nộp bản sao công chứng học bạ THPT và giấy báo trúng tuyể
- Quy định về học phí năm học 2024-2025 cho sinh viên ngành Công nghệ thông tin là 30 triệu đồng/năm. 
